<a href="https://colab.research.google.com/github/lakhanrajpatlolla/aiml-learning/blob/master/U3W13_42_Overfitting_Ants_Bees_PyTorch_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objectives

At the end of the experiment, you will be able to:

* reduce overfitting using regularization method

In [ ]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="850" height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/Walkthrough_Overfitting_Ants_Bees.mp4" type="video/mp4">
</video>
""")

## Dataset

### Description

For this experiment we have choosen a dataset which is subset of Imagenet. We have taken images belonging to ants and bees. The dataset contains 244 training images and 153 validation images.

![alt text]( https://cdn.talentsprint.com/aiml/Experiment_related_data/IMAGES/15.png)



### Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2418775" #@param {type:"string"}

In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9959000490" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import re
ipython = get_ipython()

notebook= "U3W13_42_Overfitting_Ants_Bees_PyTorch_A" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/hymenoptera_data.zip")
    ipython.magic("sx unzip /content/hymenoptera_data.zip")
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getWalkthrough() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook, "feedback_walkthrough":Walkthrough ,
              "feedback_experiments_input" : Comments,
              "feedback_inclass_mentor": Mentor_support}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


def getWalkthrough():
  try:
    if not Walkthrough:
      raise NameError
    else:
      return Walkthrough
  except NameError:
    print ("Please answer Walkthrough Question")
    return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


## Importing the required packages

In [4]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch import optim
import matplotlib.pyplot as plt

## Defining Transformation


In [6]:
image_size = (128,128)
# Define Transformation for an image
transformations = transforms.Compose([
    # YOUR CODE HERE to define transformations
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Normalize((0.5,), (0.5,))
    ])

## Data Loading


**torch.utils.data.DataLoader** class represents a Python iterable over a dataset, with following features.

1. Batching the data
2. Shuffling the data
3. Load the data in parallel using multiprocessing workers.


The batches of train and test data are provided via data loaders that provide iterators over the datasets to train our models.

In [7]:
batch_size = 100
train_set = datasets.ImageFolder('/content/hymenoptera_data/train', transform = transformations)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, num_workers=8)

# YOUR CODE HERE for Val Image folder
val_set = datasets.ImageFolder('/content/hymenoptera_data/val', transform= transformations)
# YOUR CODE HERE for Val dataloader
val_loader = torch.utils.data.DataLoader(val_set, batch_size= 100, shuffle= True, num_workers=8)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Defining the Architecture

Neural Networks are inherited from the nn.Module class.

Now let us define a neural network. Here we are using two functions \__init__ and forward function.

In the \__init__  function, we define the layers using the provided modules from the nn package. The forward function is called on the Neural Network for a set of inputs, and it passes that input through the different layers that have been defined.




In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.linear1 = nn.Linear(16384,4096)
        self.linear2 = nn.Linear(4096,1024)
        self.linear3 = nn.Linear(1024,256)
        self.linear4 = nn.Linear(256,10)
        self.linear5 = nn.Linear(10,2)

    def forward(self, x):
        # YOUR CODE HERE to implement forward pass
        print(x.size())
        out = x.view(x.shape[0], -1)
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.linear4(out)
        out = self.linear5(out)
        return out


## Calling the instances of the network

Let us declare an object of class model, and make it a CUDA model if CUDA is available:

In [20]:
# Instantiate the model
device = torch.device("cuda")
# YOUR CODE HERE to instantiate the model and convert to cuda type
model = Model().to(device)
# YOUR CODE HERE to define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr = 0.001)

## Training and Testing the model

In Training Phase, we iterate over a batch of images in the train_loader. For each batch, we perform  the following steps:

* First we zero out the gradients using zero_grad()

* We pass the data to the model i.e. we perform forward pass by calling the forward()

* We calculate the loss using the actual and predicted labels

* Perform Backward pass using backward() to update the weights

In [21]:
# No of Epochs
epoch = 20

# keeping the network in train mode
model.train()
train_losses,  train_accuracy = [], []
val_losses , val_accuracy = [], []
# Loop for no of epochs
for e in range(epoch):
    train_loss = 0
    correct = 0
    # Iterate through all the batches in each epoch
    for images, labels in trainloader:

      # Convert the image and label to gpu for faster execution
      images = images.to(device)
      labels = labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Passing the data to the model (Forward Pass)
      outputs = model(images)

      # Calculating the loss
      loss = criterion(outputs, labels)
      train_loss += loss.item()

      # Performing backward pass (Backpropagation)
      loss.backward()

      # optimizer.step() updates the weights accordingly
      optimizer.step()

      # Accuracy calculation
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        # Loop through all of the validation set
        for images, labels in val_loader:

            # YOUR CODE HERE to pass the val_images to model, calculate error and accuracy
            images = images.to(device)
            labels = labels.to(device)
            val_output = model(images)
            val_loss += criterion(val_output, labels)
            _, predicted = torch.max(val_output, 1)
            val_correct += (predicted == labels).sum().item()

    train_losses.append(train_loss/len(train_set))
    # YOUR CODE HERE to append val losses
    val_losses.append(val_loss/len(val_set))

    train_accuracy.append(100 * correct/len(train_set))
    # YOUR CODE HERE to append val accuracy
    val_accuracy.append(100 * val_correct/len(val_set))
    print('epoch: {}, Train Loss:{:.6f} Validation Loss {:.6f}Train Accuracy: {:.2f}, Validation accuracy {:.2f} '.format(e+1,train_losses[-1], val_losses[-1], train_accuracy[-1], val_accuracy[-1]))

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


torch.Size([100, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([44, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([53, 1, 128, 128])
epoch: 1, Train Loss:0.271336 Validation Loss 0.064522Train Accuracy: 47.95, Validation accuracy 56.86 
torch.Size([100, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([44, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([53, 1, 128, 128])
epoch: 2, Train Loss:0.083849 Validation Loss 0.099266Train Accuracy: 51.64, Validation accuracy 43.14 
torch.Size([100, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([44, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([53, 1, 128, 128])
epoch: 3, Train Loss:0.048268 Validation Loss 0.056258Train Accuracy: 54.51, Validation accuracy 60.13 
torch.Size([100, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([44, 1, 128, 128])
torch.Size([100, 1, 128, 128])
torch.Size([53, 1, 128, 128])
epoch: 4, Train Loss:0.047254 Validation Loss 0.037028Train Accuracy: 49.59,

## Data Augmentation



Diversity of data and a larger dataset is the easiest way to avoid overfitting of the model. Data augmentation allows you to increase the size of your dataset by performing processes like flipping, cropping, rotation, scaling and translation on the existing images. Data augmentation not only increases the dataset size but also exposes the model to different angles and lighting and reduces the bias in the dataset, thus avoiding chances of overfitting.

Added two more transformations to the original data.


*   Applied random rotation of $45^o$ using **`transforms.RandomRotation`**
*   Applied vertical flip to the images using **`transforms.RandomVerticalFlip()`**




In [22]:
image_size = (128,128)
transformations = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.Grayscale(),
                                transforms.RandomRotation(45),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                                ])

In [23]:
batch_size = 100
train_set = datasets.ImageFolder('/content/hymenoptera_data/train', transform= transformations)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, num_workers=8)


# YOUR CODE HERE to load train and validaion data in batches
val_set = datasets.ImageFolder('/content/hymenoptera_data/val', transform=transformations)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=100, shuffle=True, num_workers=8)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Regularization

Dropouts: Regularization techniques prevent the model from overfitting by modifying the cost function. Dropout, on the other hand, prevents overfitting by modifying the network itself. Every neuron apart from the ones in the output layer is assigned a probability p of being temporarily ignored from calculations. p is also called dropout rate and is initialized to 0.2. Then, as each iteration progresses, the neurons in each layer with the highest probability get dropped. This results in creating a smaller network with each epoch. Since in each iteration, a random input value can be eliminated, the network tries to balance the risk and not to favour any of the features and reduces bias and noise.

## Optimize the Architecture

In [31]:
class Optimized_Model(nn.Module):
    def __init__(self):
        super(Optimized_Model, self).__init__()

        self.linear1 = nn.Linear(16384,4096)
        self.linear2 = nn.Linear(4096,1024)
        self.linear3 = nn.Linear(1024,256)
        self.linear4 = nn.Linear(256,10)
        self.linear5 = nn.Linear(10,2)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        # YOUR CODE HERE to implement forward pass
        print('Printing x shapes')
        print(x.shape[0], x.shape[1])
        out = x.view(x.shape[0], -1)
        print('Printing out shapes After')
        print(out.shape[0], out.shape[1])
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.linear4(out)
        out = self.dropout(self.linear5(out))
        return out


## Initialize the optimized model

In [32]:
# Instantiate the model
device = torch.device("cuda")

# YOUR CODE BELOW to instantiate model and define loss function and optimizer
model2 = Optimized_Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr = 0.001)


## Training the optimized model

In Training Phase, we iterate over a batch of images in the train_loader. For each batch, we perform  the following steps:

* First we zero out the gradients using zero_grad()

* We pass the data to the model i.e. we perform forward pass by calling the forward()

* We calculate the loss using the actual and predicted labels

* Perform Backward pass using backward() to update the weights

In [33]:
# No of Epochs
epoch = 20

model2.train()
train_losses_opt,  train_accuracy_opt = [], []
val_losses_opt , val_accuracy_opt = [], []

for e in range(epoch):
    otrain_loss = 0
    ocorrect = 0
    # Iterate through all the batches in each epoch
    for images, labels in trainloader:

      # Convert the image and label to gpu for faster execution
      images = images.to(device)
      labels = labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # YOUR CODE HERE to perform forward pass
      outputs = model2(images)

      # YOUR CODE HERE to Calculate the loss
      loss = criterion(outputs, labels)
      otrain_loss += loss.item()

      # Performing backward pass (Backpropagation)
      loss.backward()

      # optimizer.step() updates the weights accordingly
      optimizer.step()

      # YOUR CODE HERE for accuracy calculation
      _, predicted = torch.max(outputs, 1)
      ocorrect = (predicted == labels).sum().item()


    oval_loss = 0
    oval_correct = 0
    with torch.no_grad():
        # Loop through all of the validation set
        for images, labels in val_loader:
            # YOUR CODE HERE to pass the val_images to model, calculate error and accuracy
            images = images.to(device)
            labels = labels.to(device)
            val_output = model2(images)
            oval_loss += criterion(val_output, labels)
            _, predicted = torch.max(val_output, 1)
            oval_correct += (predicted == labels).sum().item()


    # YOUR CODE HERE to append all train, validation accuracy and losses
    train_losses_opt.append(otrain_loss / len(train_set))
    val_losses_opt.append(oval_loss/ len(val_set))

    train_accuracy_opt.append(100 * ocorrect/ len(train_set))
    val_accuracy_opt.append(100 * oval_correct/len(val_set))

    print('epoch: {}, Train Loss:{:.6f} Test Loss {:.6f} Train Accuracy: {:.2f}, Test accuracy {:.2f} '.format(e+1,train_losses_opt[-1], val_losses_opt[-1], train_accuracy_opt[-1], val_accuracy_opt[-1]))

Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
44 1
Printing out shapes After
44 16384
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
53 1
Printing out shapes After
53 16384
epoch: 1, Train Loss:0.185806 Test Loss 0.080221 Train Accuracy: 6.97, Test accuracy 52.29 
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
44 1
Printing out shapes After
44 16384
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
53 1
Printing out shapes After
53 16384
epoch: 2, Train Loss:0.079811 Test Loss 0.127652 Train Accuracy: 9.84, Test accuracy 45.75 
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
100 1
Printing out shapes After
100 16384
Printing x shapes
44 1
Printing out shapes After
44 16384
Printing x shapes
100 1
Printing out shapes Af

### Please answer the questions below to complete the experiment:




In [42]:
#@title State True or False: Using dropout, random neurons in the layer gets deactivated at each training step { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "TRUE" #@param ["","TRUE", "FALSE"]

In [43]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [44]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "Good" #@param {type:"string"}


In [46]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [47]:
#@title  Experiment walkthrough video? { run: "auto", vertical-output: true, display-mode: "form" }
Walkthrough = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [48]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [40]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Somewhat Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [49]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 1776
Date of submission:  15 Feb 2025
Time of submission:  14:53:11
View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions
